In [ ]:
!pip install gradio

In [ ]:
import torch
import gradio as gr
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import tempfile

# Inisialisasi model
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

step = 4
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"

# Load motion adapter
adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo, ckpt), device=device))

# Load pipeline
pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe.scheduler = EulerDiscreteScheduler.from_config(
    pipe.scheduler.config,
    timestep_spacing="trailing",
    beta_schedule="linear"
)

# Fungsi utama
def generate_animation(prompt):
    try:
        output = pipe(prompt=prompt, guidance_scale=1.0, num_inference_steps=step)
        with tempfile.NamedTemporaryFile(suffix=".gif", delete=False) as tmp:
            export_to_gif(output.frames[0], tmp.name)
            gif_path = tmp.name
        return gif_path
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Gradio UI
with gr.Blocks(title="AnimateDiff Lightning - GIF Generator", css="""
    body { background-color: #f8f9fa; font-family: 'Segoe UI', sans-serif; }
    .gr-button { background-color: #4f46e5 !important; color: white !important; border-radius: 8px !important; }
    .gr-button:hover { background-color: #4338ca !important; }
""") as demo:
    gr.Markdown("""
    <div style='text-align: center; padding: 20px 0'>
        <h1 style='color: #4f46e5;'>🎞️ AnimateDiff Lightning</h1>
        <p style='font-size: 16px;'>Generate beautiful animated GIFs from text prompts using state-of-the-art diffusion models.</p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1, min_width=400):
            prompt_input = gr.Textbox(
                label="📝 Enter Your Prompt",
                placeholder="e.g. A dancing astronaut on the moon with stars in the background",
                lines=2
            )
            generate_button = gr.Button("🚀 Generate Animation")

        with gr.Column(scale=1, min_width=320):
            output_gif = gr.Image(
                label="🎬 Preview (click top-right to download)",
                type="filepath",
                width=320
            )

    generate_button.click(fn=generate_animation, inputs=[prompt_input], outputs=output_gif)

demo.launch()
